# Prelab: Genetic variation, databases and tools for analysis #

This lab has two components. 

1.	In the first part we will use an open source tool called PLINK to analyze some genetic variation data, with an emphasis on basic data quality control (QC) filtering and genotype-phenotype association testing.
2.	In the second part we will focus on various databases (UCSC Genome Browser, 1000 genomes, HapMap) to examine genetic variants. 


*** Part 1: PLINK ***

PLINK is a toolkit for whole genome association analysis. It is is very computationally efficient to do large-scale analyses. PLINK is freely available to researchers and can be downloaded from http://pngu.mgh.harvard.edu/~purcell/plink/. We have already installed the PLINK on CoCalc for you. You can open a terminal and type "plink" to run it. 

*** Summary statistics on the sample dataset ***

You will find a zip file called “data.zip” in the folder "plink_lab". Unzip this file for the following analysis.

We are ready to start exploring the data. First we want to examine and validate our data, we may simply run plink with the basic input (--file) and output (--out) commands and make sure there are no errors. The option --noweb tells the PLINK to skip the web check since we are using the command line.

In [ ]:
plink --noweb --file ./plink_lab/hapmap1 --out ./plink_lab/validate

Plink should print a bunch of information as it runs in the terminal, which will be saved to a file plink.log. You will be able to revisit this run “log” at any time by typing:

In [ ]:
more ./plink_lab/validate.log

The following are a few things to pay special attention to:

0 SNPs failed missingness test ( GENO > 1 )

This step removes individual SNPs which are missing from too many samples. Initially GENO is set to >1, so no SNP columns are removed.

0 SNPs failed frequency test ( MAF < 0 )

MAF is the Minor Allele Frequency. It can be used to exclude SNPs which are not informative because they show little variation in the sample set being analyzed. For instance, if a SNP shows variation in only 1 of the 89 individuals, it is not useful statistically and should be removed. Initially, MAF is set to < 0, meaning that no SNPs are excluded. Let’s change this to make MAF < 0.01 grounds for excluding the SNP.

We will now run plink putting explicit thresholds on SNP minor allele frequency (--maf), SNP genotyping rate (--geno) and missingness per individual/sample (--mind). Note that many missing genotypes at a SNP level are often indicative of underperforming SNP array probes, whereas many missing genotypes at an individual/sample level are usually indicative of poor sample quality. Similar to before, plink will output a log file in addition to a table of samples passing quality control filters (.imiss file) and a table of SNPs passing the filters (.lmiss file):

In [ ]:
plink --maf 0.01 --geno 0.1 --mind 0.1 --noweb --file ./plink_lab/hapmap1 --missing --out ./plink_lab/plink-filter

Geno > 0.1 criteria removes SNPs that were missing more than 10% of their genotypes across the sample set.

MAF < 0.01 criteria removes SNPs that had a Minor Allele Frequency of < 1%.

Note that these thresholds are somewhat arbitrary, but seem to be a good empirical starting point.

*** Association analysis ***

We will now perform a basic analysis, to identify associations between SNP alleles and the disease phenotype. 

Run plink using the same quality control filters as before. We will now use the --assoc command in order to tell plink to perform basic genotype-phenotype association testing:

In [ ]:
plink --maf 0.01 --geno 0.1 --mind 0.1 --noweb --file ./plink_lab/hapmap1 --assoc --out ./plink_lab/plink-filter-assoc

The association output file (.assoc) stores a single SNP association result in each row. The fields are:

Chromosome  
SNP identifier  
Location of the SNP on the chromosome (called physical distance and measured in bp)  
Code for allele 1 (the more rare or 'minor' allele based on the entire sample frequencies)  
The frequency of this variant in affected individuals (cases)  
The frequency of this variant in unaffected individuals (control)  
Code for allele 2 (the more common or 'major' allele)  
The chi-squared statistic for this test (1 df)  
The asymptotic p-value for this test  
The odds ratio for this test  

These are the most important fields. The p-value measures the statistical significance of the association of each SNP with the phenotype of interest, whereas the odds ratio measures the relative odds of falling into a given phenotype category based each SNP genotype. In this sense, the odds ratio measures the effect size of the SNP, but with an important caveat: “effect” is used liberally here, since association tests only measure correlation but do not prove causation!

Which SNPs are most strongly associated with the phenotype based on p-value? We can use the sort command to quickly sort files on the command line and find the top SNPs with the most signiciant p-values.

In [ ]:
sort -k9g ./plink_lab/plink-filter-assoc.assoc | head

Where do these p-values come from? Remember that we are looking for SNPs that are over represented in the cases compared to the controls.

<img src="chi-square.png">

Where:
mA is the # of A alleles in cases, nA is the # of A alleles in controls, mB is the # of B alleles in cases, nB is the # of B alleles in controls. M is the total number of cases, and N is the total number of controls

A chi-squared test assesses if the underlying frequencies of alleles A and B are different enough to be statistically significant.
 
Our null hypothesis is that this SNP is not associated with the disease, so our expected is that the frequency of the allele is the same in the case and controls. 

<img src="chi.png">

3. Viewing Results

Haploview is a viewer specifically for statistical genomics studies. One of its functions is to plot the results of an association analysis across the genome so that they can be visualized. Although covering Haploview goes beyond the scope of the lab (although it is easy to set up and use for anyone interested), a Haploview “Manhattan plot” (-log10 p-value vs. genomic coordinate) is shown below for the present dataset:

<img src="Haploview.png">

The highest dot represents the most significant SNP 'rs12302542' on chromosome 12. If there were a true association, we would expect some of its neighboring SNPs to also be associated with the phenotype, as in the population we would expect some of those SNP alleles to be coinherited with the rs12302542 allele. This is indeed what we see, with a number of SNPs on chromosome 12 also having significant p-value!

## Part 2: Using Genome Browser  to examine genomic variations ##

The gene MAPK3 is a member of the MAP kinase family, which is also known as extracellular signal-regulated kinases and acts in signaling cascade that regulates various cellular processes such as proliferation, differentiation, and cell cycle progression in response to a variety of extracellular signals. MAPK3 is activated by upstream kinases, resulting in its translocation to the nucleus where it phosphorylates nuclear targets. We are going to use the UCSC Genome Browser to find SNPs in this gene.

First, go to the UCSC Genome Browser and find the locus of the MAPK3 gene in human (make sure you select hg19 assembly). You will discover that several genes are called MAPK3. Click on the first isoform of the gene.

Jump to the (chr16:30,128,215-30,128,325) 3rd  exon of MAPK3. Scroll down to the Variation and Repeats section, and set the "All SNPs (138)", “1000G Ph1 Vars” and “HapMap SNPs” options to "full.”  Also, under Comparative Genomics, select “full’ for Conservation. You are welcome to turn off other tracks to have a clear and less cluttered view. Click refresh.

You will see many SNPs in this region. By looking at “Mammal Cons” scores at these SNPs, you can tell which SNPs are conserved.
We can grab all the SNPs in a region using the Tables function. Make the browser window jump back to the entire MAPK3 gene region then click on the Tables link.

Set the group option to "Variation and Repeats." The track option should be set to "All SNPs (138)" and the region options should be set to "position" and chr16:30,125,426-30,134,289, which forms the boundaries of the MAPK3 gene. Click on the "get output" button at the bottom of the screen. Notice that insertions and deletions are also returned using this technique.

### Appendix ###

We do not have the time to cover the dbSNP (http://www.ncbi.nlm.nih.gov/SNP/ ) and 1000 genomes (http://browser.1000genomes.org/index.html ) datasets, separately in the lab. 

However if you find a variant, and you want to look at it in more detail, then you must consult the dbSNP database for it. It has tons on information on the population genetics/variation aspect of the SNP, and other relevant stuff.

1000 genomes browser has the most up to date information on the genetic variation present on a particular region of the genome. You can also use this to look at specific variation in a particular population or even further sub-classify variants by the frequency.